In [1]:
import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

from math import sin, radians, sqrt

import ipywidgets as widgets

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_defaults(axes=True, grid=True, axes0=True, timeit=False, cad_width=640, height=800)

set_sidecar("Hotend", init=True)
replay_box = widgets.Checkbox(False, description='Enable Replay')
out = widgets.Output()
display(replay_box)
display(out)

@out.capture()
def changed(e):
    if e["new"]:
        enable_replay(warning=False)
    else:
        disable_replay()

replay_box.observe(changed, names='value')

Checkbox(value=False, description='Enable Replay')

Output()

In [3]:
bearing_space = 38
bearing_diameter = 15
bearing_length = 45

# https://wiki.e3d-online.com/images/b/b5/V6-175-SINK.pdf
e3d_sink_d1 = 16
e3d_sink_d2 = 12
e3d_sink_h1 = 3.7
e3d_sink_h2 = 6
e3d_sink_h3 = 3

hotend_y_offset = -3

min_thickness = 3

body_x = bearing_length
body_y = bearing_space+bearing_diameter+min_thickness*2
body_z = 40

body_b_y = bearing_diameter+min_thickness*2
body_b_z = bearing_diameter+min_thickness*2

belt_y1 = 16
belt_y2 = 9
belt_spacing = 20

In [4]:
e3d_sink_void = (
    cq.Workplane("XY")
    .cylinder(body_z, e3d_sink_d2/2)
    .faces("<Z")
    .cylinder(e3d_sink_h3, e3d_sink_d1/2, centered=(True, True, False))
    .faces("<Z[1]")
    .workplane(offset = e3d_sink_h2)
    .cylinder(e3d_sink_h1, e3d_sink_d1/2, centered=(True, True, False))
)

replay(e3d_sink_void)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [5]:
belt_width = 1.38 # GT2 belt
combined_belt_width = 2
belt_height = 5

belt_holder1_void_raw = (
    cq.Workplane("XY")
    .line(0, 2)
    .line(2, -2+combined_belt_width/2)
    .line(body_x/2, 0)
    .line(0, -combined_belt_width/2)
    .mirrorX()
    .extrude(belt_height)
)

belt_holder1_void = (
    belt_holder1_void_raw
    .translate((e3d_sink_d2/2, belt_spacing/2, body_z/2-belt_y1))
) + (
    belt_holder1_void_raw
    .mirror("YZ")
    .translate((-e3d_sink_d2/2, belt_spacing/2, body_z/2-belt_y2))
)

replay(belt_holder1_void)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [6]:
directions = [d+a for d in "<>" for a in "XYZ"]
edges = [d1+" and "+d2 for d1 in directions for d2 in directions if d1[1] < d2[1]]

select_box_edges = " or ".join(edges)
print(select_box_edges)

<X and <Y or <X and <Z or <X and >Y or <X and >Z or <Y and <Z or <Y and >Z or >X and <Y or >X and <Z or >X and >Y or >X and >Z or >Y and <Z or >Y and >Z


In [7]:
body_box = (
    cq.Workplane("XY")
    .box(body_x, body_y, body_z)
)

In [8]:
body_splitter1 = (
    body_box.faces("<X")
    .vertices(">Y and <Z")
    .workplane(centerOption="CenterOfMass")
    .rect(body_b_y, body_b_z, centered=False)
    .cutThruAll()
)

replay(body_splitter1)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [9]:
endstop_l = 13
endstop_w = 6
endstop_h = 11

body = (
    body_box
    .faces(">X")
    .workplane()
    .center(0, -body_z/2)
    .center(0, bearing_diameter/2)
    .center(0, min_thickness)
    .pushPoints([(-bearing_space/2, 0), (bearing_space/2, 0)])
    .circle(bearing_diameter/2)
    .cutThruAll()
    ## ENDSTOP HOLE
    .faces("<X")
    .workplane()
    .rect(endstop_w, endstop_l)
    .cutBlind(-endstop_h)
    ## SINK HOLE
    .faces(">Z")
    .workplane(centerOption="CenterOfMass")
    .cut(e3d_sink_void.translate((0, hotend_y_offset, 0)))
    ## HODEND SETUP HOLE
    .faces(">Z")
    .edges(">Y")
    .workplane(centerOption="CenterOfMass", invert = True)
    .rect(e3d_sink_d2, body_y/2-hotend_y_offset, centered=(True, False))
    .cutBlind(body_z-body_b_z)
    ## BELT HOLDER
    .faces(">Z")
    .workplane(centerOption="CenterOfBoundBox", invert = True)
    .pushPoints([(0, belt_spacing/2)])
    .rect(body_x, 1)
    .cutBlind(belt_y1)
    .cut(belt_holder1_void)
    ## CHAMFER
    .edges(select_box_edges)
    .chamfer(1)
)
    
replay(body)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [10]:
(remaining_body1, body_b) = (
    body.split(body_splitter1).solids().all()
)

show(body_b)

Done, using side car 'Hotend'


In [12]:
hotend_screw_d = 3
hotend_screw_head_d = 5
hotend_screw_head_h = 4
hotend_screw_z = e3d_sink_h3 + e3d_sink_h2/2
hotend_screw_x = e3d_sink_d2/2 + hotend_screw_d/2 + 1

body_c_y = body_y/2 - body_b_y - hotend_y_offset

remaining_body2 = (
    remaining_body1
    .faces("<Y[3]")
    .edges("<Z")
    .workplane(centerOption="CenterOfMass")
    .pushPoints([(hotend_screw_x, hotend_screw_z), (-hotend_screw_x, hotend_screw_z)])
    .cboreHole(hotend_screw_d, hotend_screw_head_d, hotend_screw_head_h, depth=body_c_y)
)

replay(remaining_body2)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [13]:
body_splitter2 = (
    body_splitter1
    .faces("<X")
    .workplane(offset=-endstop_h)
    .rect(body_b_y+body_c_y, body_b_z, centered=False)
    .cutBlind(-body_x)
    .faces("<X")
    .workplane()
    .rect(body_b_y+body_c_y-endstop_w, body_b_z, centered=False)
    .cutBlind(-body_x)
)

replay(body_splitter2)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'


In [14]:
(remaining_body3, body_c) = (
    remaining_body2.split(body_splitter2).solids().all()
)

show(body_c)

Done, using side car 'Hotend'


In [15]:
remaining_body4 = (
    remaining_body3
)

replay(remaining_body4)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Hotend'
